## Imports and notebook definition

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt

In [5]:
import numpy as np
import pandas as pd

In [40]:
from ecephys.data import paths, channel_groups
from ecephys.sglx_utils import load_timeseries
from ecephys.signal.csd import get_kcsd
from ecephys.signal.sharp_wave_ripples import detect_sharp_waves, get_durations, get_midpoints, get_sink_amplitudes, get_sink_integrals
from ecephys.helpers.utils import store_df_h5
import ecephys.plot as eplt

In [28]:
CONDITION = "REC"
SUBJECT = "Doppio"

## Load the data

In [8]:
sr_chans = channel_groups.stratum_radiatum_140um_to_200um[SUBJECT]
hpc_chans = channel_groups.hippocampus[SUBJECT]
bin_path = paths.get_datapath(subject=SUBJECT, condition=CONDITION, data="lf.bin")

In [9]:
(time, hpc_lfps, fs) = load_timeseries(bin_path, hpc_chans, start_time=None, end_time=None)

nChan: 385, nFileSamp: 18000019


### Explore LFPs (optional)

In [11]:
eplt.lfp_explorer(time, hpc_lfps, chan_labels=hpc_chans)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

## Detect sharp waves

In [16]:
interestimate_distance = intersite_distance = 0.020
k = get_kcsd(hpc_lfps, gdx=interestimate_distance, do_lcurve=True)

Performing L-curve parameter estimation...
No lambda given, using defaults
min lambda 1e-12
max lambda 0.0126
min lambda 1e-12
max lambda 0.0126
l-curve (all lambda):  0.23
Best lambda and R =  0.0003822395851068327 ,  0.23


In [22]:
hpc_csd = k.values('CSD')

In [25]:
sr_csd = hpc_csd[np.isin(hpc_chans, sr_chans)]

In [30]:
spws = detect_sharp_waves(time, sr_csd)

In [60]:
fig, ax1 = plt.subplots(figsize=(20, 3))
plt.hist(spws.start_time, bins=int(7200/10))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Compute SPW properties

In [35]:
spws["duration"] = get_durations(spws)
spws["midpoint"] = get_midpoints(spws)

In [36]:
spws["sink_amplitude"] = get_sink_amplitudes(spws, time, sr_csd) * (1e-6) # Scale to mA/mm

In [37]:
spws["sink_integral"] = get_sink_integrals(spws, time, fs, sr_csd) * (1e-6) * (1e3) # Scale to mA * ms

### Export results

In [288]:
metadata = dict(
    csd_chans=hpc_chans,
    detection_chans=sr_chans,
    electrode_positions=ele_pos,
    intersite_distance=intersite_distance,
    gdx=interestimate_distance,
    lambd = k.lambd,
    R = k.R,
    detect_states=["Wake", "N1", "N2", "REM"],
    detection_zscore_threshold=detection_threshold,
    boundary_zscore_threshold=boundary_threshold,
    minimum_duration=minimum_duration,
)
spws_path = paths.get_datapath(subject=SUBJECT, condition=CONDITION, data="sharp_waves.h5")
store_df_h5(spws_path, spws, **metadata)

# Detect bout-by-bout

In [62]:
from ecephys.scoring import load_visbrain_hypnogram

hypnogram = load_visbrain_hypnogram(paths.get_datapath(subject=SUBJECT, condition=CONDITION, data="hypnogram.txt"))

In [66]:
fig, ax1 = plt.subplots(figsize=(24, 3))
plt.hist(spws.start_time, bins=int(7200/10))
eplt.plot_hypnogram_overlay(hypnogram, ax=ax1)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [90]:
def get_bout_spws(time, sr_csd, bout):
    bout_mask = np.logical_and(time >= bout['start_time'], time <= bout['end_time'])
    bout_spws = detect_sharp_waves(time[bout_mask], sr_csd[:, bout_mask])
    bout_spws['state'] = bout['state']
    return bout_spws

def get_spws_bout_by_bout(time, sr_csd, hypnogram):
    spws = pd.DataFrame(columns=["spw_number", "start_time", "end_time"])
    spws.set_index('spw_number', inplace = True)
    for index, row in hypnogram.iterrows(): 
        bout_spws = get_bout_spws(time, sr_csd, row)
        spws = spws.append(bout_spws)
        
    return spws
        

In [105]:
bspws = get_spws_bout_by_bout(time, sr_csd, hypnogram)
bspws["duration"] = get_durations(bspws)
bspws["midpoint"] = get_midpoints(bspws)

In [106]:
fig, ax1 = plt.subplots(figsize=(24, 3))
plt.hist(bspws.start_time, bins=int(7200/10))
eplt.plot_hypnogram_overlay(hypnogram, ax=ax1)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [110]:
rem_spws = bspws[bspws['state'] == "REM"].reset_index()

In [115]:
from pathlib import Path

metadata = dict(
    csd_chans=hpc_chans,
    detection_chans=sr_chans,
    intersite_distance=intersite_distance,
    gdx=interestimate_distance,
    lambd = k.lambd,
    R = k.R,
    detect_states=["Wake", "N1", "N2", "REM"],
    detection_zscore_threshold=2.5,
    boundary_zscore_threshold=1,
    minimum_duration=0.005,
)
rem_spws_path = Path('/Volumes/neuropixel/Data/CNPIX4-Doppio/3-18-2020_g0_t3.imec0.rem_spws.h5')
store_df_h5(rem_spws_path, rem_spws, **metadata)

In [116]:
eplt.lazy_spw_explorer(rem_spws, metadata, SUBJECT, CONDITION)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()